In [1]:
import pandas as pd
from datetime import datetime
import time
from tqdm import tqdm

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import sys

import logging
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [2]:
import psycopg2

def create_connection():
    connection = psycopg2.connect(
        host="introduction-01-intro-ap-southeast-1-dev-introduction-db.cpfm8ml2cxp2.ap-southeast-1.rds.amazonaws.com",
        database="postgres",
        user="postgres",
        password="postgres123"
    )
    connection.set_session(autocommit=True)
    return connection

connection = create_connection()
logger.info(f"connection log: {connection}")

INFO:root:connection log: <connection object at 0x000001F064EA5470; dsn: 'user=postgres password=xxx dbname=postgres host=introduction-01-intro-ap-southeast-1-dev-introduction-db.cpfm8ml2cxp2.ap-southeast-1.rds.amazonaws.com', closed: 0>


In [3]:
event = {
    "urls" : [
        "https://phimmoiiii.net/the-loai/phim-chieu-rap",
        "https://phimmoiiii.net/phim-bo",
        "https://phimmoiiii.net/phim-le"
    ],
    "table_name": "ai4e_movie",
    "data_field": [
        {
            "field": "id",
            "type": "integer"
        },
        {
            "field": "movie_link",
            "type": "varchar"
        },
        {
            "field": "poster_image_link",
            "type": "varchar"
        },
        {
            "field": "movie_status",
            "type": "varchar"
        },
        {
            "field": "en_name",
            "type": "varchar"
        },
        {
            "field": "vn_name",
            "type": "varchar"
        },
        {
            "field": "category",
            "type": "varchar"
        }
    ]
}

In [4]:
# Step 2: Tạo table trên postgres dựa vào table anme
def create_table(connection, table_name, fields):
    logger.info(f"Start create table {table_name}")
    list_field_name_field_type = [f'{field["field"]} {field["type"]}' for field in fields] 
    result_field_name_field_type = ','.join(list_field_name_field_type)

    create_table_str = f"""
        create table if not exists {table_name} (
            {result_field_name_field_type}
        )
    """

    cursor = connection.cursor()
    cursor.execute(create_table_str)
    logger.info(f"Created table successfully. Hauct")
table_name = "ai4e_movie_ynn_hauct"
fields = event["data_field"]

create_table(connection, table_name, fields)

INFO:root:Start create table ai4e_movie_ynn_hauct
INFO:root:Created table successfully. Hauct


In [5]:
def connect_to_web(url):
    ## Set up Edge webdriver options.
    options = webdriver.EdgeOptions()
    options.headless = True # Runs driver without opening a chrome browser.
    options.add_experimental_option("excludeSwitches", ["enable-logging"])

    ## Set the path to your webdriver
    service = Service('msedgedriver.exe')

    ## Initiate the browser
    driver = webdriver.Edge(service=service, options=options)
    driver.get(url)
    driver.refresh()
    return driver

In [6]:
driver = connect_to_web(event['urls'][1])

In [7]:
event['urls'][1]+'/page/'+str(2)

'https://phimmoiiii.net/phim-bo/page/2'

In [8]:
url = event['urls'][1]

In [13]:
def crawl_data_one_page(soup, url):
    category = url.split('/')[-1]
    if (category == 'phim-chieu-rap')|(category == 'phim-le'):
        movies = soup.find_all("article", {"class": "item movies"})
    if category == 'phim-bo':    
        movies = soup.find_all("article", {"class": "item tvshows"})
        
    movies_dict = []
    
    for movie in movies:
        row = {
            "movie_id": movie['id'],
            "movie_link": movie.find('a')['href'],
            "poster_image_link": movie.find('img')['src'],
            "movie_status": movie.find('div', {'class':'trangthai'}).text if movie.find('div', {'class':'trangthai'}) else None,
            "vn_name": movie.find('img')['alt'],
            "en_name": movie.find('span').text.strip(' '),
            "category": category
        }
        movies_dict.append(row)
    movies_df = pd.DataFrame(movies_dict)
    return movies_df

In [21]:
def crawl_data(driver, url):
    print('='*30)
    print(f'Started crawling in web: {url}')
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, 'lxml')
    df0 = crawl_data_one_page(soup, url)
    print('Finished crawling page: 1')
    page_num = len(soup.find_all("div", {"class": "pagination"})[0].find_all('a', {'class': 'inactive'})) + 2
    for page_id in range(2, page_num):
        driver = connect_to_web(url+'/page/'+str(page_id))
        page_source = driver.page_source
        soup = BeautifulSoup(page_source, 'lxml')
        df1 = crawl_data_one_page(soup, url)
        df0 = pd.concat([df0, df1], ignore_index=True)
        print(f'Finished crawling page: {page_id}')
    return df0

In [ ]:
soup = BeautifulSoup(page_source, 'lxml')
print(soup.prettify())

In [ ]:
driver.find_element(By.XPATH, '//*[@id="contenedor"]/div[3]/div[1]/div[3]/a').click()

In [ ]:
pages = soup.find_all("div", {"class": "pagination"})[0].find_all('a', {'class': 'inactive'})
pages

In [ ]:
link = driver.find_element(By.XPATH, '//a[@href="https://phimmoiiii.net/phim-bo/page/3"]')
link.click()

In [ ]:
a

In [ ]:
'<span>Hoon Payon 2023 </span>'

In [ ]:
movies = soup.find_all("article", {"class": "item movies"})
movies

In [ ]:
event['urls'][0].split('/')[-1]

In [ ]:
def crawl_data_one_page(soup):
    movies = soup.find_all("article", {"class": "item tvshows"})    
    ### Convert to a pandas dataframe
    movies_dict = []
    print('Creating a dataframe')
    
    for movie in movies:
        row = {
            "movie_id": movie['id'],
            "movie_link": movie.find('a')['href'],
            "poster_image_link": movie.find('img')['src'],
            "movie_status": movie.find('div', {'class':'trangthai'}).text if movie.find('div', {'class':'trangthai'}) else None,
            "vn_name": movie.find('img')['alt'],
            "en_name": movie.find('span').text.strip(' '),
            "category": event['urls'][0].split('/')[-1]
        }
        movies_dict.append(row)    
    movies_df = pd.DataFrame(movies_dict)
    return movies_df

In [ ]:
movies_df = crawl_data_one_page(soup)
movies_df.head()

In [ ]:
url = event['urls'][0]

In [ ]:
pages = soup.find_all("div", {"class": "pagination"})
pages

In [ ]:
def click_change_page(soup):
    

In [ ]:
# Step 2: Tạo table trên postgres dựa vào table anme
def create_table(connection, table_name, fields):
    logger.info(f"Start create table {table_name}")
    list_field_name_field_type = [f'{field["field"]} {field["type"]}' for field in fields] 
    result_field_name_field_type = ','.join(list_field_name_field_type)

    create_table_str = f"""
        create table if not exists {table_name} (
            {result_field_name_field_type}
        )
    """

    cursor = connection.cursor()
    cursor.execute(create_table_str)


table_name = "ai4e_movie_ynn"
fields = event["data_field"]

create_table(connection, table_name, fields)

Kéo dữ liệu từ trang web sau và inset dữ liệu vào postgresdb
- https://phimmoiyyy.net/the-loai/phim-chieu-rap
- table name: ai4e_movies_{account}
- column name: 
    + id - integer
    + movie_link - varchar
    + poster_image_link - varchar
    + movie_name - varchar
    + movie_status - varchar
    + en_name - varchar
    + vn_name - varchar
    + category - varchar

In [ ]:
def ingest_to_db(df):
    # Connect to PostgreSQL
    conn = psycopg2.connect(
        host=HOST,
        database=DB_NAME,
        user=USER,
        password=PASSWORDS
    )

    # Create a cursor object to interact with the database
    cur = conn.cursor()

    # Create table if it doesn't exist
    create_table_query = f"""
    CREATE TABLE IF NOT EXISTS {game}.{game}_app_reviews_agg (
        review_date DATE,
        review_star VARCHAR(255),
        emotion VARCHAR(255),
        summary_review TEXT,
        num_review_star INT2,
        num_emotion INT2,
        num_summary_review INT2,
        PRIMARY KEY (review_date, review_star, emotion, summary_review)
    )
    """
    cur.execute(create_table_query)
    print('Created table')
    conn.commit()
